In [56]:
import pymongo
import gridfs
import codecs
from IPython.display import display, HTML, IFrame
import json
import redis

In [5]:
client = pymongo.MongoClient("mongodb://localhost:26000/")

## Collect Information about the database 

In [6]:
mydb = client["db"]

print(client.list_database_names())

print(mydb.list_collection_names())

['admin', 'config', 'db']
['users', 'articles', 'reads', 'articlesci']


## Making a query as an example

In [43]:
# helper methods 
def count_col(col):
    count_users = mydb[col].count_documents({})
    print(col + " count = ", count_users)


In [44]:
mycol = mydb["users"]
user1 = mycol.find_one({'id':'u12'},{'name'})
print(user1)  
count_col("users")

{'_id': ObjectId('5fde1857dc7cad1b30d3fb10'), 'name': 'user0'}
users count =  5


## Insert a document 

- #### Insert to the user collection

In [61]:
new_user = {"timestamp": "1506328859788", "id": "u12", "uid": "25",
            "name": "user0", "gender": "female", "email": "email0",
            "phone": "phone0", "dept": "dept4", "grade": "grade1",
            "language": "zh", "region": "Beijing", "role": "role0",
            "preferTags": "tags27", "obtainedCredits": "17"}

user_id = mydb.user.insert_one(new_user).inserted_id
print(user_id)
user_count()

5fde1c9cdc7cad1b30d3fb19
user count =  10


In [ ]:
user_id = mydb.user.find(new_user)

In [65]:
with open("sample_users", "r") as sample_users:
    sample_users = sample_users.readlines()
new_users =  []

for user in sample_users:
    new_users.append(json.loads(user.strip()))
new_users

[{'timestamp': '1506328859002',
  'id': 'u28',
  'uid': '2',
  'name': 'user2',
  'gender': 'male',
  'email': 'email2',
  'phone': 'phone2',
  'dept': 'dept0',
  'grade': 'grade2',
  'language': 'zh',
  'region': 'Beijing',
  'role': 'role0',
  'preferTags': 'tags13',
  'obtainedCredits': '17'},
 {'timestamp': '1506328859002',
  'id': 'u39',
  'uid': '2',
  'name': 'user2',
  'gender': 'male',
  'email': 'email2',
  'phone': 'phone2',
  'dept': 'dept0',
  'grade': 'grade2',
  'language': 'zh',
  'region': 'Beijing',
  'role': 'role0',
  'preferTags': 'tags13',
  'obtainedCredits': '17'},
 {'timestamp': '1506328859002',
  'id': 'u25',
  'uid': '2',
  'name': 'user2',
  'gender': 'male',
  'email': 'email2',
  'phone': 'phone2',
  'dept': 'dept0',
  'grade': 'grade2',
  'language': 'zh',
  'region': 'Beijing',
  'role': 'role0',
  'preferTags': 'tags13',
  'obtainedCredits': '17'}]

In [66]:
mydb.users.insert_many(new_users)

## Updating Collections


In [87]:
myquery = { "id": "u900" }
newvalues = { "$set": { "name": "old_name" } }

print("Before update\n")
for doc in mydb.users.find(myquery, {"id", "name"}):
    print(doc)

print("\n\n")

print("After update\n")

mydb.users.update_many(myquery, newvalues)

for doc in mydb.users.find(myquery, {"id", "name"}):
    print(doc)

Before update

{'_id': ObjectId('5fde1857dc7cad1b30d3fb10'), 'id': 'u900', 'name': 'new_name'}
{'_id': ObjectId('5fde1860dc7cad1b30d3fb11'), 'id': 'u900', 'name': 'new_name'}
{'_id': ObjectId('5fde1864dc7cad1b30d3fb12'), 'id': 'u900', 'name': 'new_name'}
{'_id': ObjectId('5fde1869dc7cad1b30d3fb13'), 'id': 'u900', 'name': 'new_name'}
{'_id': ObjectId('5fde1954dc7cad1b30d3fb14'), 'id': 'u900', 'name': 'new_name'}
{'_id': ObjectId('5fde1a50dc7cad1b30d3fb15'), 'id': 'u900', 'name': 'new_name'}
{'_id': ObjectId('5fde1c9cdc7cad1b30d3fb19'), 'id': 'u900', 'name': 'new_name'}
{'_id': ObjectId('5fde1c9edc7cad1b30d3fb1c'), 'id': 'u900', 'name': 'new_name'}



After update

{'_id': ObjectId('5fde1857dc7cad1b30d3fb10'), 'id': 'u900', 'name': 'old_name'}
{'_id': ObjectId('5fde1860dc7cad1b30d3fb11'), 'id': 'u900', 'name': 'old_name'}
{'_id': ObjectId('5fde1864dc7cad1b30d3fb12'), 'id': 'u900', 'name': 'old_name'}
{'_id': ObjectId('5fde1869dc7cad1b30d3fb13'), 'id': 'u900', 'name': 'old_name'}
{'_id': 

## Read and display image from GridFS

In [ ]:
# using caching on user table 

In [19]:
myclient = pymongo.MongoClient("mongodb://localhost:26000/") # connection
mydb = myclient["db"] # database name
fs = gridfs.GridFS(mydb) # connect to GridF
images = mydb['fs.files'].find({}) # find the image (input image name if neccesary)
image = fs.get(images[0]["_id"]) # get the first image
byte_data = image.read() 
base64_data = codecs.encode(byte_data, 'base64') # convert the byte data
img = base64_data.decode('utf-8') 
display(HTML('<img src=\"data:image/png;base64,'+ img + '\" alt=\"image\" />')) # display in HTML

## Caching using redis server

In [109]:
r = redis.Redis()
user_id = "u900"
result = r.get(user_id)
if(result is None):
    
    result = mydb.users.find_one({'id': 'u900'}, {"name"})
    print(result)
    r.mset({user_id: result['name']})

In [110]:
r.get(user_id)

b'old_name'